In [144]:
import pandas as pd
import gradio as gr


def get_BFCs(results_df):
    df = results_df[['id', 'commit_hash', 'is_bug_fixing_commit']]
    df = df.rename(columns={'commit_hash': 'hash', 'is_bug_fixing_commit': 'BFC'})
    df['hash'] = df['hash'].str.slice(0, 10)
    return df

results_A = pd.read_csv('130_results_michel.csv')
results_B = pd.read_csv('130_results_abhishek.csv')
results_C = pd.read_csv('130_results_david.csv')

BFCs_A = get_BFCs(results_A)
BFCs_B = get_BFCs(results_B)
BFCs_C = get_BFCs(results_C)

BFCs = pd.merge(BFCs_A, BFCs_B, on='hash', how='inner', suffixes=('A', 'B'))

BFCs = pd.merge(BFCs, BFCs_C, on='hash', how='inner', suffixes=('', 'C'))
BFCs = BFCs[['hash', 'BFCA', 'BFCB', 'BFC']]
BFCs = BFCs.rename(columns={'BFC': 'BFCC'})

In [146]:
def get_annotation(df, hash):
    """Produce markdown for the annotation of a given short hash
    in the annotation dataframe
    """
    rows = df.loc[df['commit_hash'].str.startswith(hash)]
    row = rows.iloc[0].to_dict()
    hash_long = row['commit_hash']
    link = f"https://git.kernel.org/pub/scm/linux/kernel/git/torvalds/linux.git/commit/?h=v6.7&id={hash}"
    is_bfc = row['is_bug_fixing_commit']
    is_obvious = row['is_obvious_bug']
    is_safety = row['is_safety_related']
    comment = row['comment']
    visited = row['link_visited']
    result = (f"[{hash}]({link})\n* bfc: {is_bfc}\n* obvious: {is_obvious}\n"
              f"* safety: {is_safety}\n* visited: {visited}\n"
              f"* comment: {comment}")
    return result
port =10020

In [147]:
with gr.Blocks() as ui:
    with gr.Row():
        filter_ddw = gr.Dropdown(choices=[("All", "all"),
                                  ("All annotators equal", "all-equal"),
                                  ("Any annotator different", "any-dif"),
                                  ("Annotators A, B different", "ab-dif"),
                                  ("Annotators B, C different", "bc-dif"),
                                  ("Annotators A, C different", "ac-dif")],
                                value="all")
        count_md = gr.Markdown(f"Commits: {len(BFCs.index)}")
    bfcs_df = gr.Dataframe(value=BFCs, height=300)
    with gr.Row():
        A_md = gr.Markdown()
        B_md = gr.Markdown()
        C_md = gr.Markdown()

    @filter_ddw.change(inputs=[filter_ddw], outputs=[bfcs_df, count_md])
    def filter_records(choice):
        if choice == "all":
            df = BFCs
        elif choice == "all-equal":
            df = BFCs.query("(BFCA == BFCB) and (BFCB == BFCC)")
        elif choice == "any-dif":
            df = BFCs.query("(BFCA != BFCB) or (BFCB != BFCC)")
        elif choice == "ab-dif":
            df = BFCs.query("BFCA != BFCB")
        elif choice == "bc-dif":
            df = BFCs.query("BFCB != BFCC")
        elif choice == "ac-dif":
            df = BFCs.query("BFCA != BFCC")
        count = len(df.index)
        return df, f"Commits: {count}"
        
    def select_commit(event: gr.SelectData, bfcs):
        #print(event.index, event.target, event.value, bfcs)
        row = bfcs.iloc[event.index[0]]
        hash = row['hash']
        annot_A = get_annotation(results_A, hash)
        annot_B = get_annotation(results_B, hash)
        annot_C = get_annotation(results_C, hash)
        return "**A**\n\n" + annot_A, "**B**\n\n" + annot_B, "**C**\n\n" + annot_C
        
    bfcs_df.select(select_commit, [bfcs_df], [A_md, B_md, C_md])

port = port + 1
ui.launch(server_port=port)

Running on local URL:  http://127.0.0.1:10021

To create a public link, set `share=True` in `launch()`.
